In [1]:
import math
import openai
import os
import sklearn.metrics
import random
from tqdm import tqdm
from langchain.llms import OpenAI

In [2]:
import baseten
baseten.login("yiVOous9.2mjkPJSPdya6FUFbGKmXaLyUF6ZxTQYs")

INFO API key set.
INFO:baseten:API key set.


In [3]:
from task_utils import TASKS, load_data, load_prompt, generate_prompts

In [8]:
## your LLM stack goes here

# Flan T5 XL

model = baseten.deployed_model_version_id('qvvgl9q')

def call_llm(prompt):
    output = model.predict({"prompt": prompt})
    result = output["data"][0]
    return result
    

In [9]:
call_llm("what is capital of usa?")

'Washington, D.C.'

In [10]:
def evaluate(tasks: list, tasks_dir: str):
    report = dict()
    for task in tqdm(tasks):
        train_df, test_df = load_data(task=task, tasks_dir=tasks_dir)
        prompt_template = load_prompt(prompt_name="base_prompt.txt", task=task, tasks_dir=tasks_dir)
        prompts = generate_prompts(prompt_template=prompt_template, data_df=train_df)
        report[task] = dict()
        targets = list()
        outputs = list()
        for prompt, data in zip(prompts, train_df.iterrows()):
            datapoint_id, data = data
            output = call_llm(prompt)
            output = output.strip()
            targets.append(data['answer'])
            outputs.append(output)
            success = output == data['answer']
            report[task][datapoint_id] = {
                'prompt': prompt,
                'generated_output': output,
                'correct_output': data['answer'],
                'success': output == data['answer']
            }
        report[task]['balanced_accuracy'] = sklearn.metrics.balanced_accuracy_score(targets, outputs)
    
    print('Balanced Accuracy:', sum([report[task]['balanced_accuracy'] if not math.isnan(report[task]['balanced_accuracy']) else 0 for task in tasks])/len(tasks))
    
    return report


In [11]:
len(TASKS)

162

In [12]:
tasks_dir = '../legalbench'

# tasks=random.sample(TASKS, 10)

tasks= [task for task in TASKS if task.startswith("cuad")]

In [13]:
len(tasks)

38

In [14]:
report = evaluate(tasks=tasks, tasks_dir=tasks_dir)

100%|██████████| 38/38 [02:14<00:00,  3.53s/it]

Balanced Accuracy: 0.7587719298245613


In [15]:
print('Balanced Accuracy:', sum([report[task]['balanced_accuracy'] if not math.isnan(report[task]['balanced_accuracy']) else 0 for task in tasks])/len(tasks))

Balanced Accuracy: 0.7587719298245613
